# SQL para Engenharia de Dados e Ciência de Dados - Aula 4: Filtros e Seleção Condicional

## **Trabalhando com Filtros AND, OR, IN e BETWEEN e Comando CASE**

Este notebook explora o uso de filtros avançados e seleção condicional em SQL para manipular e analisar dados de forma eficiente. As práticas incluem comandos AND, OR, IN, BETWEEN, LIKE e o uso do CASE para criar seleções condicionais.

---

### **Seleção Condicional**

Caso tenhamos diversas condições e, para cada uma delas, queremos retornar uma seleção diferente, podemos utilizar o comando `CASE` associado com `WHEN`, `THEN` e `ELSE` (opcional).

**Sintaxe do CASE:**

```sql
CASE 
  WHEN <condicao1> THEN <comando1>
  WHEN <condicao2> THEN <comando2>
  ...
  WHEN <condicaoN> THEN <comandoN>
  ELSE <comando_caso_nenhuma_condicao_nao_seja_satisfeita>
END
```

### **Tradução literal do fluxo:**

1. Caso a `<condicao1>` seja satisfeita, execute o `<comando1>` e finalize.
2. Caso a `<condicao1>` não seja satisfeita, avalie a `<condicao2>`.
3. Caso a `<condicao2>` seja satisfeita, execute o `<comando2>` e finalize.
4. Continue avaliando as condições até encontrar uma satisfeita.
5. Caso nenhuma condição seja satisfeita, execute o comando definido no `ELSE` (opcional).

O comando `CASE` pode ser utilizado dentro de comandos como `SELECT`, `UPDATE`, `DELETE` ou de forma isolada.

### **Exemplo de Uso:**

```sql
SELECT id_cliente, id_loja, valor,
CASE
    WHEN valor > 1000 THEN 'Compra com alto valor'
    WHEN valor < 1000 THEN 'Compra com baixo valor'
END 
AS classeValor 
FROM transacoes 
WHERE valor > 50;
```

**Resultado esperado:**

| id_cliente | valor   | id_loja     | classeValor             |
|------------|---------|-------------|-------------------------|
| 1          | 50.74   | magalu      | Compra com baixo valor  |
| 3          | 110.00  | postoshell  | Compra com baixo valor  |
| 1          | 2000.90 | magalu      | Compra com alto valor   |
| 4          | 50.74   | extra       | Compra com baixo valor  |
| 3          | 1100.00 | shopee      | Compra com alto valor   |

Agora, vamos explorar esses conceitos e outros comandos práticos como `AND`, `OR`, `IN`, `BETWEEN` e `LIKE`.

---

## **1. Configuração da Tabela no AWS Athena**

A tabela `transacoes` será utilizada para os exercícios práticos. Caso ainda não tenha configurado os dados no S3, siga os passos abaixo:

1. Crie uma pasta `bucket-transacoes` no seu S3 e carregue o arquivo `transacoes.csv`.
2. No AWS Athena, execute o seguinte comando para criar a tabela:

```sql
CREATE EXTERNAL TABLE transacoes(
    id_cliente BIGINT,
    id_transacao BIGINT,
    valor FLOAT,
    id_loja STRING
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES ('separatorChar' = ',', 'quoteChar' = '"', 'escapeChar' = '\\')
STORED AS TEXTFILE
LOCATION 's3://bucket-transacoes/';
```

---

## **2. Trabalhando com Filtros**

### **Query 1: Usando AND**

```sql
SELECT *
FROM transacoes
WHERE valor > 30 AND id_loja = 'magalu';
```

**Resultado:**

```markdown
| id_cliente | id_transacao | valor   | id_loja  |
|------------|--------------|---------|----------|
| 1          | 768805383    | 50.74   | magalu   |
| 1          | 76856563     | 2000.90 | magalu   |
```

---

### **Query 2: Usando OR**

```sql
SELECT *
FROM transacoes
WHERE valor > 30 OR id_loja = 'magalu';
```

**Resultado:**

```markdown
| id_cliente | id_transacao | valor   | id_loja     |
|------------|--------------|---------|-------------|
| 1          | 768805383    | 50.74   | magalu      |
| 2          | 768805399    | 30.90   | giraffas    |
| 3          | 818770008    | 110.00  | postoshell  |
| 1          | 76856563     | 2000.90 | magalu      |
| 4          | 764545534    | 50.74   | extra       |
| 3          | 8154567758   | 1100.00 | shopee      |
```

---

### **Query 3: Usando IN**

```sql
SELECT *
FROM transacoes
WHERE id_loja IN ('magalu', 'subway') AND valor > 10;
```

**Resultado:**

```markdown
| id_cliente | id_transacao | valor   | id_loja  |
|------------|--------------|---------|----------|
| 1          | 768805383    | 50.74   | magalu   |
| 1          | 76856563     | 2000.90 | magalu   |
| 1          | 767573759    | 15.70   | subway   |
```

---

### **Query 4: Usando BETWEEN**

```sql
SELECT *
FROM transacoes
WHERE valor BETWEEN 60.0 AND 1000.0;
```

**Resultado:**

```markdown
| id_cliente | id_transacao | valor  | id_loja     |
|------------|--------------|--------|-------------|
| 3          | 818770008    | 110.0  | postoshell  |
```

---

## **3. Trabalhando com LIKE e Padrões**

### **Query 5: Usando LIKE com Padrão Inicial**

```sql
SELECT * 
FROM transacoes 
WHERE id_loja LIKE 'mag%';
```

**Resultado:**

```markdown
| id_cliente | id_transacao | valor   | id_loja  |
|------------|--------------|---------|----------|
| 1          | 768805383    | 50.74   | magalu   |
| 1          | 76856563     | 2000.90 | magalu   |
```

---

### **Query 6: Usando LIKE com Padrão Contido**

```sql
SELECT * 
FROM transacoes 
WHERE id_loja LIKE '%sh%';
```

**Resultado:**

```markdown
| id_cliente | id_transacao | valor   | id_loja     |
|------------|--------------|---------|-------------|
| 3          | 818770008    | 110.0   | postoshell  |
| 3          | 8154567758   | 1100.0  | shopee      |
```

---

## **4. Trabalhando com CASE e Seleção Condicional**

### **Query 7: Combinando CASE e SELECT**

```sql
SELECT id_cliente, id_loja, valor,
CASE
    WHEN valor > 1000 THEN 'Compra com alto valor'
    WHEN valor < 1000 THEN 'Compra com baixo valor'
END 
AS classeValor, 
CASE
    WHEN id_loja IN ('giraffas', 'subway') THEN 'alimentacao'
    WHEN id_loja IN ('magalu', 'extra') THEN 'variedade'
    WHEN id_loja IN ('postoshell', 'seveneleven') THEN '24horas'
    ELSE 'outros'
END 
AS tipo_compra
FROM transacoes;
```

**Resultado:**

```markdown
| id_cliente | id_loja     | valor   | classeValor           | tipo_compra    |
|------------|-------------|---------|-----------------------|----------------|
| 1          | magalu      | 50.74   | Compra com baixo valor| variedade      |
| 2          | giraffas    | 30.90   | Compra com baixo valor| alimentacao    |
| 3          | postoshell  | 110.00  | Compra com baixo valor| 24horas        |
| 1          | magalu      | 2000.90 | Compra com alto valor | variedade      |
| 1          | subway      | 15.70   | Compra com baixo valor| alimentacao    |
| 3          | seveneleven | 2.99    | Compra com baixo valor| 24horas        |
| 4          | extra       | 50.74   | Compra com baixo valor| variedade      |
| 5          | subway      | 10.00   | Compra com baixo valor| alimentacao    |
| 3          | shopee      | 1100.00 | Compra com alto valor | outros         |
```

---

## **Conclusão**

Neste notebook, exploramos o uso de filtros avançados (`AND`, `OR`, `IN`, `BETWEEN`, `LIKE`) e seleção condicional com o comando `CASE`. Essas técnicas são fundamentais para refinar consultas e obter insights mais detalhados a partir de dados complexos. Pratique essas operações em seus projetos para consolidar os conceitos e otimizar a análise de dados!
